<a href="https://colab.research.google.com/github/yongminkim0501/Image_deep/blob/main/Unet_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pylab as plt

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2

In [ ]:
drive_path = "/content/drive/MyDrive/"
source_filename = drive_path + "dataset/archive (1).zip"

extract_folder = "dataset/"

import shutil
shutil.unpack_archive(source_filename, extract_folder)

In [ ]:
shutil.move('test_set','/content/drive/MyDrive/dataset/data')
shutil.move('training_set','/content/drive/MyDrive/dataset/data')

In [ ]:
import os
import glob
forder_dir = "/content/drive/MyDrive/dataset"

train_dir = forder_dir + "/data/training_set/training_set/"
train_cat_dir = train_dir + "cats/"
train_dog_dir = train_dir + "dogs/"

test_dir = forder_dir + "/data/test_set/"
test_cat_dir = test_dir + "cats/"
test_dog_dir = test_dir + "dogs/"

data_dir = forder_dir + "/data"

os.chdir(data_dir)

train_jpg = []
test_jpg = []

files = os.listdir(train_cat_dir)
for file in files:
    f_dic = file.split(".")
    if f_dic[-1] == 'jpg':
      f = cv2.imread(train_cat_dir + file)
      f = cv2.resize(f, (256, 256))
      train_jpg.append(f)

files = os.listdir(train_dog_dir)
for file in files:
    f_dic = file.split(".")
    if f_dic[-1] == 'jpg':
      f = cv2.imread(train_dog_dir + file)
      f = cv2.resize(f, (256, 256))
      train_jpg.append(f)

files = os.listdir(test_cat_dir)
for file in files:
    f_dic = file.split(".")
    if f_dic[-1] == 'jpg':
      f = cv2.imread(test_cat_dir + file)
      f = cv2.resize(f, (256, 256))
      test_jpg.append(f)

files = os.listdir(test_dog_dir)
for file in files:
    f_dic = file.split(".")
    if f_dic[-1] == 'jpg':
      f = cv2.imread(test_dog_dir + file)
      f = cv2.resize(f, (256, 256))
      test_jpg.append(f)

In [ ]:
train_img_1 = []
valid_img_1 = []
test_img_1 = []
y_valid = []
y_train = []
y_test = []
for i in range(350):
  train_img_1.append(train_jpg[i])
  train_img_1.append(train_jpg[5000+i])
  y_train.append(0)
  y_train.append(1)

for i in range(150):
  valid_img_1.append(train_jpg[i+400])
  valid_img_1.append(train_jpg[5400+i])
  y_valid.append(0)
  y_valid.append(1)

for i in range(250):
  test_img_1.append(test_jpg[i])
  test_img_1.append(test_jpg[1000+i])
  y_test.append(0)
  y_test.append(1)

x_valid = np.array(valid_img_1)
x_train = np.array(train_img_1)
x_test = np.array(test_img_1)

y_train = np.array(y_train)
y_test = np.array(y_test)
y_valid = np.array(y_valid)

In [ ]:
x_valid = x_valid / 255.0
x_train = x_train / 255.0
x_test = x_test / 255.0


In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input, Flatten,Dense
from tensorflow.keras.models import Model

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    size = 8
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, size * 4)
    s2, p2 = encoder_block(p1, size * 8)
    s3, p3 = encoder_block(p2, size * 16)
    s4, p4 = encoder_block(p3, size * 32)

    b1 = conv_block(p4, size * 64)

    d1 = decoder_block(b1, s4, size * 32)
    d2 = decoder_block(d1, s3, size * 16)
    d3 = decoder_block(d2, s2, size * 8)
    d4 = decoder_block(d3, s1, size * 4)

    flat = Flatten()(d4)

    outputs = Dense(1, activation="sigmoid")(flat)

    model = Model(inputs, outputs, name="U-Net")
    return model

In [ ]:
input_shape = (256, 256, 3)
model = build_unet(input_shape)
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=["accuracy"])
history = model.fit(x_train, y_train ,validation_data = (x_valid, y_valid), epochs=10)

Model: "U-Net"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 256, 256, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_37 (Conv2D)        │ (None, 256, 256, 32)   │            896 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_36    │ (None, 256, 256, 32)   │            128 │ conv2d_37[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_36             │ (None, 256, 256, 32)   │              0 │ batch_normalization_3… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_38 (Conv2D)        │ (None, 256, 256, 32)   │          9,248 │ activation_36[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_37    │ (None, 256, 256, 32)   │            128 │ conv2d_38[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_37             │ (None, 256, 256, 32)   │              0 │ batch_normalization_3… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_8           │ (None, 128, 128, 32)   │              0 │ activation_37[0][0]    │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_39 (Conv2D)        │ (None, 128, 128, 64)   │         18,496 │ max_pooling2d_8[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_38    │ (None, 128, 128, 64)   │            256 │ conv2d_39[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_38             │ (None, 128, 128, 64)   │              0 │ batch_normalization_3… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_40 (Conv2D)        │ (None, 128, 128, 64)   │         36,928 │ activation_38[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_39    │ (None, 128, 128, 64)   │            256 │ conv2d_40[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_39             │ (None, 128, 128, 64)   │              0 │ batch_normalization_3… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_9      

 Total params: 9,868,993 (37.65 MB)

 Trainable params: 9,863,105 (37.62 MB)

 Non-trainable params: 5,888 (23.00 KB)

Epoch 1/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 158s 4s/step - accuracy: 0.5010 - loss: 128.0494 - val_accuracy: 0.5000 - val_loss: 273478.8750
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 71s 667ms/step - accuracy: 0.5167 - loss: 20.6826 - val_accuracy: 0.5000 - val_loss: 242.4585
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 21s 672ms/step - accuracy: 0.5014 - loss: 4.1405 - val_accuracy: 0.5000 - val_loss: 447.7227
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 20s 659ms/step - accuracy: 0.5813 - loss: 1.0905 - val_accuracy: 0.5000 - val_loss: 2.8145
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 21s 699ms/step - accuracy: 0.5391 - loss: 0.9280 - val_accuracy: 0.5000 - val_loss: 2.7751
Epoch 6/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 20s 664ms/step - accuracy: 0.6574 - loss: 0.6586 - val_accuracy: 0.5000 - val_loss: 0.7647
Epoch 7/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 21s 706ms/step - accuracy: 0.7384 - loss: 0.5098 - val_accuracy: 0.5000 - val_loss: 0.7969
Epoch 8/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 20s 659ms/step - accuracy: 0.8084 - loss: 0.4489 